# COGS 108 - Data Checkpoint

# Names

- Victoria Thai
- Hannah Yick
- Jane Dinh
- Natasha Supangkat
- Gabriel Ramiro

<a id='research_question'></a>
# Research Question

What trends can be found between a county’s designation of a superfund site and its socioeconomic/demographic trends? Does the demographic makeup of a county influence the amount of time between Superfund designation and the completion of the remediation process?

# Dataset(s)

Superfund/National Priorities List (NPL) Sites 
- Dataset Name: U.S. EPA SUPERFUND PROGRAM: All Final NPL Sites
- Link to the dataset: https://semspub.epa.gov/work/HQ/201371.pdf 
- Number of observations: 1327 (as of April 26, 2021)
- Features: Region, state, site name, site ID, EPA ID, address, city, zip, county, federal facility indicator (whether or not the      site is a federal site), latitude, longitude, listing date.
    - We will be focusing on the site name, county, and listing date.
- Summary: This dataset provides a comprehensive overview of the main characteristics of current Superfund sites on the National       Priorities List. Geographical information (county) can help us better understand the spread of the data by location, and will        allow us to combine with demographic data to explore trends. Additionally, we will use the listing date to determine the source of   funding (based on the date of policy changes) as well as to measure how long the site has been active.

Demographic Data
- Dataset Name:
- Link to the dataset:
- Number of observations:
- Features:
- Summary:

We will combine our datasets using the common variable of geographic location (county). By analyzing both aspects - Superfund activity and demographic breakdown - holistically, we will be able to better visualize and understand trends and potentially a relationship between the two.


# Setup

In [1]:
#Imports 
import pandas as pd
import numpy as np

#Graphing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#Statistics
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

#Webscraping
import requests 
import bs4
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION